In [4]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import mean_squared_error
import random

# Definisikan data (berdasarkan gambar yang diunggah)
data = {
    'Suhu': [20, 25, 16, 15, 27, 12, 19, 28, 35, 20],
    'Kelembaban': [16, 12, 30, 14, 10, 19, 12, 8, 6, 17],
    'Durasi': [55.7, 59.3, 5.6, 30.1, 43.4, 18.6, 22.1, 47.3, 76.4, 20.9]
}

# Membagi data menjadi set pelatihan dan pengujian
indeks_latih = list(range(8))
indeks_uji = list(range(8, 10))

X_latih = np.array([[data['Suhu'][i], data['Kelembaban'][i]] for i in indeks_latih])
y_latih = np.array([data['Durasi'][i] for i in indeks_latih])

X_uji = np.array([[data['Suhu'][i], data['Kelembaban'][i]] for i in indeks_uji])
y_uji = np.array([data['Durasi'][i] for i in indeks_uji])

# Fungsi untuk inisialisasi individu
def inisialisasi_individu():
    # Inisialisasi individu dengan nilai acak untuk parameter fuzzy
    suhu_rendah = sorted([random.uniform(0, 20), random.uniform(10, 20), random.uniform(15, 30)])
    suhu_sedang = sorted([random.uniform(15, 25), random.uniform(20, 30), random.uniform(25, 35)])
    suhu_tinggi = sorted([random.uniform(25, 40), random.uniform(30, 40), random.uniform(35, 50)])
    
    kelembaban_rendah = sorted([random.uniform(0, 20), random.uniform(10, 30), random.uniform(20, 40)])
    kelembaban_sedang = sorted([random.uniform(20, 40), random.uniform(30, 50), random.uniform(40, 60)])
    kelembaban_tinggi = sorted([random.uniform(40, 60), random.uniform(50, 80), random.uniform(60, 100)])
    
    durasi_pendek = sorted([random.uniform(0, 25), random.uniform(10, 35), random.uniform(25, 50)])
    durasi_panjang = sorted([random.uniform(50, 75), random.uniform(60, 80), random.uniform(75, 100)])
    
    return (suhu_rendah, suhu_sedang, suhu_tinggi, kelembaban_rendah, kelembaban_sedang, kelembaban_tinggi, durasi_pendek, durasi_panjang)

# Fungsi untuk membuat sistem fuzzy
def buat_sistem_fuzzy(param):
    # Unpack the params untuk fungsi keanggotaan fuzzy
    (suhu_rendah, suhu_sedang, suhu_tinggi, kelembaban_rendah, kelembaban_sedang, kelembaban_tinggi, durasi_pendek, durasi_panjang) = param
    
    # Definisikan variabel fuzzy
    suhu = ctrl.Antecedent(np.arange(0, 41, 1), 'suhu')
    kelembaban = ctrl.Antecedent(np.arange(0, 101, 1), 'kelembaban')
    durasi = ctrl.Consequent(np.arange(0, 101, 1), 'durasi')  # Memperbaiki rentang durasi

    # Fungsi keanggotaan untuk suhu
    suhu['rendah'] = fuzz.trimf(suhu.universe, suhu_rendah)
    suhu['sedang'] = fuzz.trimf(suhu.universe, suhu_sedang)
    suhu['tinggi'] = fuzz.trimf(suhu.universe, suhu_tinggi)
    
    # Fungsi keanggotaan untuk kelembaban
    kelembaban['rendah'] = fuzz.trimf(kelembaban.universe, kelembaban_rendah)
    kelembaban['sedang'] = fuzz.trimf(kelembaban.universe, kelembaban_sedang)
    kelembaban['tinggi'] = fuzz.trimf(kelembaban.universe, kelembaban_tinggi)
    
    # Fungsi keanggotaan untuk durasi
    durasi['pendek'] = fuzz.trimf(durasi.universe, durasi_pendek)
    durasi['panjang'] = fuzz.trimf(durasi.universe, durasi_panjang)

    # Definisikan aturan fuzzy
    aturan1 = ctrl.Rule(suhu['rendah'] & kelembaban['rendah'], durasi['panjang'])
    aturan2 = ctrl.Rule(suhu['sedang'] & kelembaban['sedang'], durasi['pendek'])
    aturan3 = ctrl.Rule(suhu['tinggi'] & kelembaban['tinggi'], durasi['pendek'])

    # Membangun sistem kontrol
    durasi_ctrl = ctrl.ControlSystem([aturan1, aturan2, aturan3])
    simulasi_durasi = ctrl.ControlSystemSimulation(durasi_ctrl)
    
    return simulasi_durasi

# Fungsi untuk algoritma genetika
def algoritma_genetika():
    # Inisialisasi populasi
    ukuran_populasi = 20
    generasi = 100
    laju_mutasi = 0.1
    laju_persilangan = 0.7
    populasi = [inisialisasi_individu() for _ in range(ukuran_populasi)]
    
    for gen in range(generasi):
        # Evaluasi fitness
        nilai_fitness = [evaluasi_fitness(individu) for individu in populasi]
        
        # Seleksi
        terpilih = seleksi(populasi, nilai_fitness)
        
        # Persilangan
        populasi_baru = []
        for i in range(0, ukuran_populasi, 2):
            orangtua1 = terpilih[i]
            orangtua2 = terpilih[i+1]
            keturunan1, keturunan2 = persilangan(orangtua1, orangtua2)
            populasi_baru.extend([keturunan1, keturunan2])
        
        # Mutasi
        for i in range(ukuran_populasi):
            if random.random() < laju_mutasi:
                populasi_baru[i] = mutasi(populasi_baru[i])
        
        populasi = populasi_baru
    
    # Evaluasi akhir
    individu_terbaik = max(populasi, key=evaluasi_fitness)
    return individu_terbaik

# Fungsi evaluasi
def evaluasi_fitness(individu):
    simulasi = buat_sistem_fuzzy(individu)
    prediksi = []
    
    for data_titik in X_latih:
        nilai_suhu, nilai_kelembaban = data_titik
        simulasi.input['suhu'] = nilai_suhu
        simulasi.input['kelembaban'] = nilai_kelembaban
        simulasi.compute()
        
        prediksi.append(simulasi.output['durasi'])  # Mengambil output durasi
    
    error = mean_squared_error(y_latih, prediksi)
    return 1 / (error + 1e-6)  # Inverse of error for fitness

# Fungsi seleksi
def seleksi(populasi, nilai_fitness):
    # Pilih individu berdasarkan nilai fitness
    terpilih = np.random.choice(populasi, size=len(populasi)//2, p=nilai_fitness/np.sum(nilai_fitness))
    return terpilih

# Fungsi persilangan
def persilangan(orangtua1, orangtua2):
    # Persilangan sederhana dengan mengambil bagian dari orangtua
    titik = random.randint(1, len(orangtua1) - 1)
    keturunan1 = orangtua1[:titik] + orangtua2[titik:]
    keturunan2 = orangtua2[:titik] + orangtua1[titik:]
    return keturunan1, keturunan2

# Fungsi mutasi
def mutasi(individu):
    # Mutasi dengan mengubah satu parameter acak
    idx = random.randint(0, len(individu) - 1)
    individu[idx] = inisialisasi_individu()[idx]  # Ganti parameter dengan yang baru
    return individu

# Eksekusi utama
param_sistem_fuzzy_terbaik = algoritma_genetika()

print("Parameter sistem fuzzy terbaik:", param_sistem_fuzzy_terbaik)

KeyError: 'durasi'